# Préparation du jeu de données PubChem - Ajout du bruit

Dans ce notebook, on va définir les fonctions créant les fichiers de données qui seront utilisées pour l'entraînement et la validation du modèle (coordonnées bruitées et labels de différences de distances attendues)

#### Définition des chemins de fichiers

In [ ]:
test_set_location = "../data/test_set_riken_v2.h5"
train_set_location = "../data/train_set_riken_v2.h5"
minimal_set_riken = "../data/minimal_set_riken_v2.h5"